In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Sapienza/DL

/content/drive/MyDrive/Sapienza/DL


In [ ]:
import pickle
import pandas as pd
import numpy as np
import json
import itertools
import gensim
path="SEMEVAL-2021-task6-corpus/data/"


In [ ]:
with open('training_task1_labels.pkl', 'rb') as handle:
    train_labs = pickle.load(handle)

with open('original_training_task1_features.pkl', 'rb') as handle:
    train_feats = pickle.load(handle)

with open('training_task1_all_labels.pkl', 'rb') as handle:
    all_labels = pickle.load(handle)

with open('dev_task1_labels.pkl', 'rb') as handle:
    dev_labs = pickle.load(handle)

with open('original_dev_task1_features.pkl', 'rb') as handle:
    dev_feats = pickle.load(handle)

with open('test_task1_labels.pkl', 'rb') as handle:
    test_labs = pickle.load(handle)

with open('original_test_task1_features.pkl', 'rb') as handle:
    test_feats = pickle.load(handle)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1, 3),
                                   stop_words='english',
                                   strip_accents='unicode',
                                   token_pattern=r'\w{1,}',
                                   lowercase=True)


In [ ]:
train_feats[:3]

['THERE ARE ONLY TWO GENDERS FEMALE  MALE',
 'This is not an accident!',
 'SO BERNIE BROS HAVENT COMMITTED VIOLENCE EH? POWER COMES FROM THE BARREL OF A GUN, COMRADES. WHAT ABOUT THE ONE WHO SHOT CONGRESSMAN SCALISE OR THE DAYTON OHIO MASS SHOOTER?']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from wordcloud import STOPWORDS


In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer

snow_stemmer = SnowballStemmer(language='english')

def apply_stemmer(text):
    words = text.split()
    sent = [snow_stemmer.stem(word) for word in words if not word in set(STOPWORDS)]
    return ' '.join(sent)

In [ ]:
train_feats1=list(map(apply_stemmer, train_feats))
train_feats1[:3]

['there are onli two gender femal male',
 'this accident!',
 'so berni bros havent commit violenc eh? power come from the barrel of a gun, comrades. what about the one who shot congressman scalis or the dayton ohio mass shooter?']

In [ ]:
dev_feats1=list(map(apply_stemmer, dev_feats))
dev_feats1[:3]

['*president* biden? please, no.',
 'joe versus the volcan kremlin don will you shut up, man? lord of the lie',
 'anti-vaxx be like... hang on a sec - just collect my fact']

In [ ]:
test_feats1=list(map(apply_stemmer, test_feats))
test_feats1[:3]

['the democrat new america',
 'we are at war! there complex assault presidenti elect multifacet tentacl terrorist organ oper 2020 terror',
 'kill himself in prison won an honest elect']

In [ ]:
X_train = count_vectorizer.fit_transform(train_feats1)
X_valid = count_vectorizer.transform(dev_feats1)
X_test = count_vectorizer.transform(test_feats1)

In [ ]:
y_train=train_labs
y_valid=dev_labs

In [ ]:
y_train[:3]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0.]])

In [ ]:
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(C = 10, penalty='l2', solver = 'liblinear', random_state=42)

one_vs_rest = OneVsRestClassifier(log_reg)
one_vs_rest.fit(X_train, y_train)

y_train_pred_proba = one_vs_rest.predict_proba(X_train)
y_valid_pred_proba = one_vs_rest.predict_proba(X_valid)

y_pred = (y_train_pred_proba > 0.3) 
y_train_new=(y_train > 0.3) 

print("Training accuracy: ", accuracy_score(y_train_new, y_pred))

y_pred1 = (y_valid_pred_proba > 0.3) 
y_dev_new=(y_valid > 0.3) 

print("Validation Accuracy: ", accuracy_score(y_dev_new, y_pred1))

from sklearn.metrics import classification_report, f1_score

f1_score(y_dev_new,y_pred1,average='weighted')

Training accuracy:  0.9956395348837209
Validation Accuracy:  0.23809523809523808


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.3586934877216207

In [ ]:
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(C = 100, penalty='l2', solver = 'liblinear', random_state=42)

one_vs_rest = OneVsRestClassifier(log_reg)
one_vs_rest.fit(X_train, y_train)

y_train_pred_proba = one_vs_rest.predict_proba(X_train)
y_valid_pred_proba = one_vs_rest.predict_proba(X_valid)

y_pred = (y_train_pred_proba > 0.3) 
y_train_new=(y_train > 0.3) 

print("Training accuracy: ", accuracy_score(y_train_new, y_pred))

y_pred1 = (y_valid_pred_proba > 0.3) 
y_dev_new=(y_valid > 0.3) 

print("Validation Accuracy: ", accuracy_score(y_dev_new, y_pred1))


f1_score(y_dev_new,y_pred1,average='weighted')

Training accuracy:  0.9956395348837209
Validation Accuracy:  0.19047619047619047


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.33668513233730624

In [ ]:
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(C = 20, penalty='l2', solver = 'liblinear', random_state=42)

one_vs_rest = OneVsRestClassifier(log_reg)
one_vs_rest.fit(X_train, y_train)

y_train_pred_proba = one_vs_rest.predict_proba(X_train)
y_valid_pred_proba = one_vs_rest.predict_proba(X_valid)

y_pred = (y_train_pred_proba > 0.3) 
y_train_new=(y_train > 0.3) 

print("Training accuracy: ", accuracy_score(y_train_new, y_pred))

y_pred1 = (y_valid_pred_proba > 0.3) 
y_dev_new=(y_valid > 0.3) 

print("Validation Accuracy: ", accuracy_score(y_dev_new, y_pred1))

f1_score(y_dev_new,y_pred1,average='weighted')

Training accuracy:  0.9956395348837209
Validation Accuracy:  0.23809523809523808


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.34256274908448825

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.3586934877216207

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

rnd = RandomForestClassifier(n_estimators=100,n_jobs=-1)

one_vs_rest = OneVsRestClassifier(rnd)
one_vs_rest.fit(X_train, y_train)

y_train_pred_proba = one_vs_rest.predict_proba(X_train)
y_valid_pred_proba = one_vs_rest.predict_proba(X_valid)

y_pred = (y_train_pred_proba > 0.3) 
y_train_new=(y_train > 0.3) 

print("Training accuracy: ", accuracy_score(y_train_new, y_pred))

y_pred1 = (y_valid_pred_proba > 0.3) 
y_dev_new=(y_valid > 0.3) 

print("Validation Accuracy: ", accuracy_score(y_dev_new, y_pred1))

f1_score(y_dev_new,y_pred1,average='weighted')

Training accuracy:  0.9723837209302325
Validation Accuracy:  0.23809523809523808


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.3447883695725739

In [ ]:
y_pred = (y_train_pred_proba > 0.4) 
y_train_new=(y_train > 0.4) 

print("Training accuracy: ", accuracy_score(y_train_new, y_pred))

y_pred1 = (y_valid_pred_proba > 0.4) 
y_dev_new=(y_valid > 0.4) 

print("Validation Accuracy: ", accuracy_score(y_dev_new, y_pred1))

f1_score(y_dev_new,y_pred1,average='weighted')

Training accuracy:  0.9956395348837209
Validation Accuracy:  0.1746031746031746


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.2885489389837215